<a href="https://colab.research.google.com/github/ohilikeit/DSF_AED/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
! pip install geopandas
! pip install pydeck

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.2 MB/s 


In [147]:
import pandas as pd
import numpy as np
import os
import re
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
os.chdir('/content/drive/MyDrive/ML_projects/DSF')

def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

df = pd.read_csv('./자동심장 충격기 정보 조회(AED)(표준 데이터).csv', encoding='cp949')
df['address'] = df['설치기관주소'].apply(lambda x: x.split(' ')[1])
df_dongdaemoon = df[df['address'] == "동대문구"]

# 1. 인구 데이터 - 행정동 별 수요점수 계산

In [15]:
pop = pd.read_csv('./동대문구_동별_연령대별_인구수.csv')

In [16]:
cols = pop.columns
pop['male'] = pop[cols[2]] * 0.01 + pop[cols[3]] * 0.026 + pop[cols[4]] * 0.039 + pop[cols[5]] * 0.075 + pop[cols[6]] * 0.139 + pop[cols[7]] * 0.7
pop['male'] = pop['male'] / pop['male'].max()
pop['female'] = pop[cols[8]] * 0.01 + pop[cols[9]] * 0.026 + pop[cols[10]] * 0.039 + pop[cols[11]] * 0.075 + pop[cols[12]] * 0.139 + pop[cols[13]] * 0.7
pop['female'] = pop['female'] / pop['female'].max()
pop['a'] = 0.64 * pop['male'] + 0.36 * pop['female']

# 2. 도보 노드 데이터

In [17]:
import geopandas as gpd
from shapely import wkt

geo_data = './서울시 자치구별 도보 네트워크 공간정보.csv'
df = gpd.read_file(geo_data, encoding="cp949")

In [19]:
df_dobo = df[(df['노드링크 유형'] == 'NODE') &
             (df['시군구명'] == '동대문구') &
             (df['육교'] == "0") &
             (df['횡단보도'] == "0")]

# 용두동 + 신설동 = 용신동
df_dobo.loc[(df_dobo['읍면동명'] == "용두동") |
            (df_dobo['읍면동명'] == "신설동"), "읍면동명"] = "용신동"
df_dobo = df_dobo[['노드 WKT', '노드 ID', '시군구명', '읍면동명', "geometry"]].reset_index(drop=True)

df_dobo['노드 WKT'] = df_dobo['노드 WKT'].apply(wkt.loads)
df_dobo.set_geometry('노드 WKT', drop=True, inplace=True, crs = "epsg:4326")

df_dobo['buffer'] = df_dobo.to_crs({'init':'epsg:5179'}).buffer(100).to_crs({'init':'epsg:4326'})
df_dobo['buffer_coordinates'] = df_dobo['buffer'].apply(polygon_to_coordinates)

df_dobo['lon'] = df_dobo['geometry'].x
df_dobo['lat'] = df_dobo['geometry'].y

# 수요 가중치(인구) 병합
df_dobo = pd.merge(df_dobo, pop[['행정동코드명', 'a']], how= 'left', right_on = '행정동코드명', left_on = '읍면동명')
del df_dobo['행정동코드명']
df_dobo

,노드 ID,시군구명,읍면동명,geometry,buffer,buffer_coordinates,lon,lat,a
0,92202,동대문구,이문동,POINT (127.05837 37.59440),"POLYGON ((127.05950 37.59440, 127.05950 37.594...","[[127.05950420905869, 37.59440409844549], [127...",127.058371,37.594400,0.591425
1,86075,동대문구,회기동,POINT (127.05407 37.59294),"POLYGON ((127.05520 37.59295, 127.05520 37.592...","[[127.05520450301294, 37.59294540271266], [127...",127.054072,37.592941,0.124781
2,86074,동대문구,회기동,POINT (127.05425 37.59274),"POLYGON ((127.05538 37.59274, 127.05538 37.592...","[[127.05538266510756, 37.59273935249894], [127...",127.054250,37.592735,0.124781
3,86069,동대문구,회기동,POINT (127.05374 37.59279),"POLYGON ((127.05487 37.59279, 127.05487 37.592...","[[127.05487364536583, 37.592792226190134], [12...",127.053741,37.592788,0.124781
4,87176,동대문구,회기동,POINT (127.04950 37.59144),"POLYGON ((127.05063 37.59145, 127.05062 37.591...","[[127.05062842067416, 37.591449045964566], [12...",127.049496,37.591445,0.124781
...,...,...,...,...,...,...,...,...,...
8536,72386,동대문구,용신동,POINT (127.03957 37.57744),"POLYGON ((127.04070 37.57745, 127.04069 37.577...","[[127.04069792095235, 37.57744655576289], [127...",127.039565,37.577442,0.479149
8537,72387,동대문구,용신동,POINT (127.03937 37.57745),"POLYGON ((127.04051 37.57745, 127.04050 37.577...","[[127.0405052683881, 37.57745119907481], [127....",127.039373,37.577447,0.479149
8538,72388,동대문구,용신동,POINT (127.03956 37.57733),"POLYGON ((127.04069 37.57734, 127.04068 37.577...","[[127.04068923607814, 37.57733899805388], [127...",127.039557,37.577335,0.479149
8539,72389,동대문구,용신동,POINT (127.03937 37.57735),"POLYGON ((127.04051 37.57735, 127.04050 37.577...","[[127.0405052138428, 37.57735050396753], [127....",127.039373,37.577346,0.479149


# 3. 기존 AED 위치 데이터(24시간 기준)

In [140]:
aed_24 = pd.read_csv("./AED동대문구_24.csv")

aed_24 = gpd.GeoDataFrame(aed_24, geometry=gpd.points_from_xy(aed_24['경도'], aed_24['위도']), crs = 'epsg:4326')

aed_24['buffer'] = aed_24.to_crs({'init':'epsg:5179'}).buffer(100).to_crs({'init':'epsg:4326'})
aed_24['buffer_coordinates'] = aed_24['buffer'].apply(polygon_to_coordinates)

aed_24 = aed_24[['설치기관주소', '설치위치', '경도', '위도', 'geometry', 'buffer', 'buffer_coordinates']]

,AED모델명,우편번호(앞자리),우편번호(뒤자리),관리책임자명,관리자연락처,설치기관주소,설치기관전화번호,설치위치,제조사,설치기관명,경도,위도,geometry,buffer,buffer_coordinates
0,HR-501,26,16,김원식,010-3766-7792,서울특별시 동대문구 답십리로56길 105 (답십리동 답십리 파크자이),02-6949-5236,관리사무실 내,(주)라디안큐바이오,답십리파크자이 아파트,127.058457,37.567790,POINT (127.05846 37.56779),"POLYGON ((127.05959 37.56779, 127.05958 37.567...","[[127.05958923965952, 37.567794369621076], [12..."
1,HR-501,24,34,조용호,010-9650-2350,서울특별시 동대문구 휘경로 60 (휘경동 휘경SK VIEW 아파트),02-2245-5800,후문경비초소,라디안,휘경SK뷰아파트,127.065445,37.594077,POINT (127.06545 37.59408),"POLYGON ((127.06658 37.59408, 127.06657 37.593...","[[127.06657793889555, 37.594080735056], [127.0..."
2,AED Plus,25,13,백남규,02-2217-6340,서울특별시 동대문구 장안벚꽃로5길 19 (휘경동 휘경베스트빌현대아파트),02-2217-6340,정문 경비실,졸메디컬,휘경베스트빌현대아파트,127.074159,37.582067,POINT (127.07416 37.58207),"POLYGON ((127.07529 37.58207, 127.07529 37.581...","[[127.07529175821415, 37.5820714803259], [127...."
3,HR501,24,90,정영선,010-3669-0945,서울특별시 동대문구 왕산로 225 미주상가 A동 (청량리동),010-3669-0945,미주상가A동 3초소 경비실,(주)라디안큐바이오,미주상가A동,127.047209,37.582123,POINT (127.04721 37.58212),"POLYGON ((127.04834 37.58213, 127.04834 37.582...","[[127.04834165496358, 37.58212703890078], [127..."
4,NT-381.C,26,22,김경진,2244-3285,서울특별시 동대문구 고미술로 93 답십리1 치안센터 (답십리동),2244-3285,출입문 옆,나눔테크,답십리1 치안센터,127.056813,37.564664,POINT (127.05681 37.56466),"POLYGON ((127.05794 37.56467, 127.05794 37.564...","[[127.05794489208839, 37.56466819509338], [127..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,NT-180,25,98,최병원,010-4190-3661,서울특별시 동대문구 답십리로 130,02-2249-8682,보안초소(3초소) 310동,(주)나눔테크,답십리래미안위브,127.052402,37.572857,POINT (127.05240 37.57286),"POLYGON ((127.05353 37.57286, 127.05353 37.572...","[[127.0535348154347, 37.57286086818036], [127...."
108,AED-2100K,25,98,최병원,010-4190-3661,서울특별시 동대문구 답십리로 130,02-2249-8682,노인정앞(108동),니혼코덴코리아,답십리래미안위브,127.052402,37.572857,POINT (127.05240 37.57286),"POLYGON ((127.05353 37.57286, 127.05353 37.572...","[[127.0535348154347, 37.57286086818036], [127...."
109,HeartPlus NT-180,24,86,김아석,010-3299-9389,서울특별시 동대문구 약령시로21길 29 동대문경찰서,961-4169,유치장 내,나눔테크,동대문경찰서 유치장,127.045794,37.585158,POINT (127.04579 37.58516),"POLYGON ((127.04693 37.58516, 127.04692 37.585...","[[127.0469269007201, 37.585161972771985], [127..."
110,HeartON A15-G4,130,731,송도섭,02-2246-0525,서울특별시 동대문구 답십리로 210-30 (답십리동 한신아파트),02-2246-0525,노인정 입구,(주)메디아나,동답한신아파트,127.063006,37.571392,POINT (127.06301 37.57139),"POLYGON ((127.06414 37.57140, 127.06413 37.571...","[[127.06413809487529, 37.571395966331444], [12..."


In [117]:
aed_24 = aed_24.iloc[:, 2:4]
aed_24.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [133]:
aed_24 = aed_24[['경도', '위도']]

pandas.core.frame.DataFrame

In [136]:
import pydeck as pdk
# layer1 = pdk.Layer(
#     'PolygonLayer',
#     aed_24,
#     get_polygon='buffer_coordinates',
#     get_fill_color= '[255, 255, 0, 130]',
#     get_fill_line=[255, 255, 255, 100],
#     pickable=True,
#     auto_highlight=True)

# layer2 = pdk.Layer(
#     'PolygonLayer',
#     points,
#     get_polygon='coordinates',
#     get_fill_color= '[0, 255, 255, 200]',
#     get_fill_line=[255, 255, 255, 100],
#     pickable=True,
#     auto_highlight=True)

layer3 = pdk.Layer(
    'ScatterplotLayer',
    aed_24,
    get_position='[경도, 위도]',
    get_radius=6,
    get_fill_color='[255, 0, 0]',
    pickable=True,
    auto_highlight=True)


center = [37.5744, 127.0400]
view_state = pdk.ViewState(
    longitude=center[0],
    latitude=center[1],
    zoom=3)

r = pdk.Deck(layers=[layer3], initial_view_state=view_state)
r.to_html()

<IPython.core.display.Javascript object>

# 4. 인접 기준 AED 위치 기반 가중치 b 생성 및 병합

In [162]:
total = pd.DataFrame()
for i in tqdm(df_dobo.index):
    db = df_dobo.loc[i, 'buffer']
    num = []
    for j in aed_24.index:
        aed = aed_24.loc[j, "buffer"]
        num.append(db.intersects(aed))
    info = {"intersects_num" : sum(num)}
    total = total.append(info, ignore_index=True)

100%|██████████| 8541/8541 [00:32<00:00, 262.87it/s]


In [170]:
total['intersects_num'] = 8 - total['intersects_num']
total['intersects_num'] = total['intersects_num'] / total['intersects_num'].max()

df_dobo['b'] = total['intersects_num'].values
df_dobo['w'] = (df_dobo['a'] + df_dobo['b']) / 2

,노드 ID,시군구명,읍면동명,geometry,buffer,buffer_coordinates,lon,lat,a,b,w
0,92202,동대문구,이문동,POINT (127.05837 37.59440),"POLYGON ((127.05950 37.59440, 127.05950 37.594...","[[127.05950420905869, 37.59440409844549], [127...",127.058371,37.594400,0.591425,1.0,0.795713
1,86075,동대문구,회기동,POINT (127.05407 37.59294),"POLYGON ((127.05520 37.59295, 127.05520 37.592...","[[127.05520450301294, 37.59294540271266], [127...",127.054072,37.592941,0.124781,1.0,0.562391
2,86074,동대문구,회기동,POINT (127.05425 37.59274),"POLYGON ((127.05538 37.59274, 127.05538 37.592...","[[127.05538266510756, 37.59273935249894], [127...",127.054250,37.592735,0.124781,1.0,0.562391
3,86069,동대문구,회기동,POINT (127.05374 37.59279),"POLYGON ((127.05487 37.59279, 127.05487 37.592...","[[127.05487364536583, 37.592792226190134], [12...",127.053741,37.592788,0.124781,1.0,0.562391
4,87176,동대문구,회기동,POINT (127.04950 37.59144),"POLYGON ((127.05063 37.59145, 127.05062 37.591...","[[127.05062842067416, 37.591449045964566], [12...",127.049496,37.591445,0.124781,1.0,0.562391
...,...,...,...,...,...,...,...,...,...,...,...
8536,72386,동대문구,용신동,POINT (127.03957 37.57744),"POLYGON ((127.04070 37.57745, 127.04069 37.577...","[[127.04069792095235, 37.57744655576289], [127...",127.039565,37.577442,0.479149,1.0,0.739575
8537,72387,동대문구,용신동,POINT (127.03937 37.57745),"POLYGON ((127.04051 37.57745, 127.04050 37.577...","[[127.0405052683881, 37.57745119907481], [127....",127.039373,37.577447,0.479149,1.0,0.739575
8538,72388,동대문구,용신동,POINT (127.03956 37.57733),"POLYGON ((127.04069 37.57734, 127.04068 37.577...","[[127.04068923607814, 37.57733899805388], [127...",127.039557,37.577335,0.479149,1.0,0.739575
8539,72389,동대문구,용신동,POINT (127.03937 37.57735),"POLYGON ((127.04051 37.57735, 127.04050 37.577...","[[127.0405052138428, 37.57735050396753], [127....",127.039373,37.577346,0.479149,1.0,0.739575


# 5. MCLP